In [265]:
# def get_results(start_index, cuisines):

def get_results(start_index):
    headers = {
        'Accept': 'application/json',
        'user-key': '4acc54ca3cf4e3c53521751bd0a2ca40',
    }

    params = (
        ('entity_id', '305'),
        ('entity_type', 'city'),
        ('start', start_index ),
#         ('cuisines', cuisines)
    )
    
    url = 'https://developers.zomato.com/api/v2.1/search'
    response = requests.get(url, headers=headers, params=params)
    return response.json()


#x = get_results(0)

all_results = []

for x in range(0, 100, 20):
    try:
        # get the data
        temp_result = get_results(x)
        all_results.append(temp_result)
        
    except Exception as e:
        print(e, x)

In [266]:
start_index = 0

headers = {
    'Accept': 'application/json',
    'user-key': '4acc54ca3cf4e3c53521751bd0a2ca40'
    }

params = (
        ('entity_id', '305'),
        ('entity_type', 'city'),
        ('cuisines', 177),
        ('start', start_index)
    )
    
url = 'https://developers.zomato.com/api/v2.1/search'
response_test = requests.get(url, headers=headers, params=params)

In [267]:
for_df = []

for row in all_results:
    for restaurant in row.get('restaurants'):
        for_df.append(restaurant.get('restaurant'))
        
    
df = pd.DataFrame.from_records(for_df)

df['res_id'] = df['R'].map(lambda x: x.get('res_id'))
df['lat'] = df['location'].map(lambda x: x.get('latitude'))
df['lng'] = df['location'].map(lambda x: x.get('longitude'))
df['aggregate_rating'] = df['user_rating'].map(lambda x: x.get('aggregate_rating'))
df['rating_text'] = df['user_rating'].map(lambda x: x.get('rating_text'))
df['votes'] = df['user_rating'].map(lambda x: x.get('votes'))
# df['cuisines'] = df['location'].map(lambda x: x.get('zipcode'))

keep = ['res_id', 'name', 'lat','lng','cuisines','aggregate_rating','rating_text','votes']
newdf = df[keep].copy()

newdf.head()

newdf

,res_id,name,lat,lng,cuisines,aggregate_rating,rating_text,votes
0,16971711,Snooze,39.7556527778,-104.9889972222,American,4.7,Excellent,642
1,16967791,Cherry Cricket,39.7194800000,-104.9567300000,"American, Burger, Mexican",4.7,Excellent,865
2,16971985,Steuben's Food Service,39.7433166667,-104.9804694444,American,4.7,Excellent,689
3,16972224,Sushi Den,39.6895611111,-104.9804833333,"Japanese, Sushi",4.8,Excellent,571
4,16972712,Vesta,39.7525500000,-104.9964100000,New American,4.6,Excellent,461
5,16973602,Root Down,39.7631694444,-105.0056138889,"American, Tapas",4.8,Excellent,480
6,16974750,Linger,39.7595972222,-105.0114444444,"American, Asian, European",4.4,Very Good,362
7,16971385,Rioja,39.7477638889,-104.9993138889,Mediterranean,4.5,Excellent,426
8,16973071,Osteria Marco,39.7476700000,-104.9995300000,Italian,4.9,Excellent,446
9,16974402,Euclid Hall,39.7477200000,-104.9999900000,"Bar Food, American",4.5,Excellent,382
